# Gemini Function Calling - 조작 중심 실습

**학습 목표**
- 미리 제공된 함수로 Function Calling 실행
- 자동/수동 호출 흐름 체험
- 응답에서 function_call 추출 및 처리

**참고 자료**
- [Gemini Function Calling 공식 문서](https://ai.google.dev/gemini-api/docs/function-calling)

---
## 1. 환경 설정 + 함수 로드

In [ ]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
print("✅ Client 연결 완료")

In [ ]:
# ============================================================
# 📦 준비물: 미리 정의된 함수 10개
# ============================================================
# 이 함수들은 실제 API를 호출하지 않고 mock 데이터를 반환합니다.
# Function Calling의 흐름을 학습하는 데 집중하세요.

def get_weather(city: str) -> dict:
    """주어진 도시의 현재 날씨를 조회합니다.
    
    Args:
        city: 도시 이름 (예: "서울", "부산", "도쿄")
    
    Returns:
        온도, 상태, 습도를 포함한 날씨 정보
    """
    weather_data = {
        "서울": {"temp": 15, "condition": "맑음", "humidity": 45},
        "부산": {"temp": 18, "condition": "흐림", "humidity": 60},
        "도쿄": {"temp": 20, "condition": "비", "humidity": 80},
        "뉴욕": {"temp": 10, "condition": "눈", "humidity": 70},
    }
    return weather_data.get(city, {"temp": 20, "condition": "알 수 없음", "humidity": 50})


def get_stock_price(symbol: str) -> dict:
    """주식 종목의 현재 가격을 조회합니다.
    
    Args:
        symbol: 주식 심볼 (예: "AAPL", "GOOGL", "005930")
    
    Returns:
        현재가, 변동률, 거래량 정보
    """
    stock_data = {
        "AAPL": {"price": 178.50, "change": 2.3, "volume": 45000000},
        "GOOGL": {"price": 141.20, "change": -0.8, "volume": 22000000},
        "005930": {"price": 71500, "change": 1.5, "volume": 12000000},  # 삼성전자
    }
    return stock_data.get(symbol, {"price": 0, "change": 0, "volume": 0})


def calculate_tip(amount: float, percentage: int) -> dict:
    """식사 금액에 대한 팁을 계산합니다.
    
    Args:
        amount: 식사 금액 (원 또는 달러)
        percentage: 팁 비율 (10, 15, 20 등)
    
    Returns:
        팁 금액과 총액
    """
    tip = amount * (percentage / 100)
    return {"tip": tip, "total": amount + tip}


def search_products(query: str, max_results: int = 5) -> dict:
    """상품을 검색합니다.
    
    Args:
        query: 검색어
        max_results: 최대 결과 수 (기본값: 5)
    
    Returns:
        검색된 상품 목록
    """
    products = [
        {"name": f"{query} 상품 {i+1}", "price": 10000 * (i+1), "rating": 4.0 + i*0.2}
        for i in range(max_results)
    ]
    return {"results": products, "total_count": max_results}


def convert_currency(amount: float, from_currency: str, to_currency: str) -> dict:
    """통화를 변환합니다.
    
    Args:
        amount: 변환할 금액
        from_currency: 원래 통화 (예: "USD", "KRW", "JPY")
        to_currency: 변환할 통화
    
    Returns:
        변환된 금액과 환율
    """
    rates = {
        ("USD", "KRW"): 1320,
        ("KRW", "USD"): 0.00076,
        ("USD", "JPY"): 149,
        ("JPY", "USD"): 0.0067,
    }
    rate = rates.get((from_currency, to_currency), 1)
    return {"converted": amount * rate, "rate": rate}


def get_time(timezone: str) -> dict:
    """특정 시간대의 현재 시간을 조회합니다.
    
    Args:
        timezone: 시간대 (예: "Asia/Seoul", "America/New_York")
    
    Returns:
        현재 시간과 날짜
    """
    from datetime import datetime
    # Mock: 실제로는 pytz 등을 사용
    return {"time": "14:30:00", "date": "2025-01-15", "timezone": timezone}


def translate_text(text: str, target_language: str) -> dict:
    """텍스트를 번역합니다.
    
    Args:
        text: 번역할 텍스트
        target_language: 목표 언어 (예: "ko", "en", "ja")
    
    Returns:
        번역된 텍스트
    """
    # Mock 번역
    translations = {
        "ko": f"[한국어 번역] {text}",
        "en": f"[English] {text}",
        "ja": f"[日本語] {text}",
    }
    return {"translated": translations.get(target_language, text)}


def calculate_bmi(weight: float, height: float) -> dict:
    """BMI(체질량지수)를 계산합니다.
    
    Args:
        weight: 체중 (kg)
        height: 키 (cm)
    
    Returns:
        BMI 값과 판정
    """
    height_m = height / 100
    bmi = weight / (height_m ** 2)
    if bmi < 18.5:
        status = "저체중"
    elif bmi < 25:
        status = "정상"
    elif bmi < 30:
        status = "과체중"
    else:
        status = "비만"
    return {"bmi": round(bmi, 1), "status": status}


def set_reminder(message: str, minutes: int) -> dict:
    """리마인더를 설정합니다.
    
    Args:
        message: 리마인더 메시지
        minutes: 몇 분 후에 알림
    
    Returns:
        설정된 리마인더 정보
    """
    return {"status": "설정됨", "message": message, "alert_in": f"{minutes}분 후"}


def get_news(category: str, count: int = 3) -> dict:
    """카테고리별 뉴스를 조회합니다.
    
    Args:
        category: 뉴스 카테고리 (예: "tech", "sports", "business")
        count: 가져올 뉴스 개수
    
    Returns:
        뉴스 목록
    """
    news = [
        {"title": f"{category} 뉴스 {i+1}", "source": "Mock News", "time": "1시간 전"}
        for i in range(count)
    ]
    return {"news": news, "category": category}


# 함수 목록 확인
AVAILABLE_FUNCTIONS = {
    "get_weather": get_weather,
    "get_stock_price": get_stock_price,
    "calculate_tip": calculate_tip,
    "search_products": search_products,
    "convert_currency": convert_currency,
    "get_time": get_time,
    "translate_text": translate_text,
    "calculate_bmi": calculate_bmi,
    "set_reminder": set_reminder,
    "get_news": get_news,
}

print(f"✅ {len(AVAILABLE_FUNCTIONS)}개 함수 로드 완료:")
for name in AVAILABLE_FUNCTIONS:
    print(f"  - {name}")

---
## 2. 자동 함수 호출 (Automatic Function Calling)

Python 함수를 직접 `tools` 파라미터에 전달하면, SDK가 자동으로:
1. 함수 스키마를 생성하고
2. 모델의 함수 호출 요청을 감지하고
3. 함수를 실행하고
4. 결과를 모델에 전달하여 최종 응답을 생성합니다.

**핵심:** 개발자는 함수만 정의하면 됨. 나머지는 SDK가 처리.

In [ ]:
# 자동 함수 호출 예시 (제공됨)
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="서울 날씨 어때?",
    config=types.GenerateContentConfig(
        tools=[get_weather]  # Python 함수 직접 전달
    )
)

print("🤖 자동 호출 결과:")
print(response.text)

In [ ]:
# TODO(human): 다양한 프롬프트로 자동 함수 호출 테스트
# 목적: 자동 함수 호출이 어떻게 작동하는지 체험
# 힌트: tools에 여러 함수를 리스트로 전달 가능
# 예상: 프롬프트에 맞는 함수가 자동으로 선택되어 실행됨

# 테스트할 프롬프트들:
# - "애플 주가 알려줘"
# - "50000원 식사에 15% 팁은 얼마야?"
# - "내 BMI 계산해줘. 몸무게 70kg, 키 175cm"

test_prompt = "애플 주가 알려줘"  # 원하는 프롬프트로 변경

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=test_prompt,
    config=types.GenerateContentConfig(
        # TODO: tools에 적절한 함수들 추가
        tools=[]
    )
)

print(f"📝 프롬프트: {test_prompt}")
print(f"🤖 응답: {response.text}")

---
## 3. 수동 함수 호출 (Manual Function Calling)

자동 호출을 비활성화하면, 모델이 "이 함수를 호출하고 싶다"는 요청만 반환합니다.  
개발자가 직접 함수를 실행하고 결과를 다시 전달해야 합니다.

**사용 시나리오:**
- 함수 실행 전 유효성 검사 필요
- 사용자 확인이 필요한 경우
- 외부 API 호출 로깅/모니터링

In [ ]:
# 수동 함수 호출 설정
config = types.GenerateContentConfig(
    tools=[get_weather, get_stock_price, calculate_tip],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(
        disable=True  # 자동 호출 비활성화
    )
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="부산 날씨 알려줘",
    config=config
)

print("🔍 응답 분석:")
print(f"  - text 속성: {response.text}")
print(f"  - function_calls: {response.function_calls}")

In [ ]:
# function_call 추출 및 실행 (제공됨)
if response.function_calls:
    for fn_call in response.function_calls:
        print(f"📞 호출 요청:")
        print(f"  - 함수명: {fn_call.name}")
        print(f"  - 인자: {fn_call.args}")
        
        # 함수 실행
        func = AVAILABLE_FUNCTIONS.get(fn_call.name)
        if func:
            result = func(**fn_call.args)
            print(f"  - 실행 결과: {result}")

In [ ]:
# TODO(human): 함수 실행 결과를 모델에 다시 전달하여 최종 응답 받기
# 목적: 수동 함수 호출의 전체 흐름 이해
# 힌트: 
#   1. types.Part.from_function_response(name=함수명, response=결과)
#   2. 대화 기록(contents)에 모델 응답과 함수 결과 추가
#   3. 다시 generate_content 호출
# 예상: 모델이 함수 결과를 바탕으로 자연어 응답 생성

# 1단계: 초기 요청
contents = [types.Content(role="user", parts=[types.Part(text="부산 날씨 알려줘")])]

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=config
)

# 2단계: 함수 호출 추출 및 실행
if response.function_calls:
    fn_call = response.function_calls[0]
    func = AVAILABLE_FUNCTIONS.get(fn_call.name)
    result = func(**fn_call.args)
    
    print(f"✅ 함수 실행 완료: {result}")
    
    # 3단계: 대화 기록 업데이트
    # TODO: contents에 모델의 function_call 응답 추가
    # TODO: contents에 함수 실행 결과 추가 (from_function_response 사용)
    
    # 4단계: 최종 응답 받기
    # final_response = client.models.generate_content(...)
    # print(f"🤖 최종 응답: {final_response.text}")

---
## 4. tool_config 모드 실험

### `types.ToolConfig` - 함수 호출 동작 제어

모델이 함수를 언제, 어떻게 호출할지 세밀하게 제어합니다.

**모드 종류:**

| 모드 | 동작 |
|-----|------|
| `AUTO` | 모델이 자율적으로 결정 (기본값) |
| `ANY` | 반드시 함수 중 하나를 호출 |
| `NONE` | 함수 호출 금지, 텍스트만 응답 |

In [ ]:
# 모드별 동작 비교 함수
def test_tool_config_mode(mode: str, prompt: str):
    """tool_config 모드별 동작을 테스트합니다."""
    config = types.GenerateContentConfig(
        tools=[get_weather, get_stock_price],
        automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True),
        tool_config=types.ToolConfig(
            function_calling_config=types.FunctionCallingConfig(
                mode=mode
            )
        )
    )
    
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
        config=config
    )
    
    return {
        "mode": mode,
        "text": response.text,
        "function_calls": response.function_calls
    }

In [ ]:
# AUTO 모드: 모델이 판단
result = test_tool_config_mode("AUTO", "서울 날씨 어때?")
print("🔹 AUTO 모드 (함수 호출 필요한 질문):")
print(f"  - text: {result['text']}")
print(f"  - function_calls: {result['function_calls']}")

In [ ]:
# AUTO 모드: 함수 불필요한 질문
result = test_tool_config_mode("AUTO", "안녕하세요")
print("🔹 AUTO 모드 (함수 호출 불필요한 질문):")
print(f"  - text: {result['text']}")
print(f"  - function_calls: {result['function_calls']}")

In [ ]:
# TODO(human): ANY 모드와 NONE 모드 테스트
# 목적: 각 모드별 동작 차이 확인
# 힌트: test_tool_config_mode 함수 활용
# 예상:
#   - ANY: "안녕하세요"에도 함수 호출 시도
#   - NONE: "서울 날씨"에도 함수 호출 없이 텍스트 응답

# ANY 모드 테스트
# result_any = test_tool_config_mode("ANY", "안녕하세요")
# print(...)

# NONE 모드 테스트
# result_none = test_tool_config_mode("NONE", "서울 날씨 어때?")
# print(...)

---
## 5. 병렬 함수 호출

하나의 프롬프트에서 여러 함수를 동시에 호출할 수 있습니다.  
예: "서울과 부산 날씨 비교해줘" → get_weather 2번 호출

In [ ]:
# 병렬 호출 테스트
config = types.GenerateContentConfig(
    tools=[get_weather, get_stock_price],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True),
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="ANY")
    )
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="서울과 도쿄 날씨 비교해줘",
    config=config
)

print(f"📞 함수 호출 요청 {len(response.function_calls)}개:")
for i, fn_call in enumerate(response.function_calls, 1):
    print(f"  {i}. {fn_call.name}({fn_call.args})")

In [ ]:
# TODO(human): 병렬 호출된 모든 함수 실행 후 결과 모아서 모델에 전달
# 목적: 복수 함수 호출 처리 방법 학습
# 힌트:
#   1. for문으로 각 function_call 실행
#   2. 각 결과를 types.Part.from_function_response로 변환
#   3. 모든 결과를 하나의 Content에 담아 전달
# 예상: 모델이 두 도시 날씨를 비교하는 응답 생성

# 여기에 코드 작성

---
## 6. 종합 실습: 챗봇 시뮬레이션

In [ ]:
# TODO(human): 모든 함수를 활용한 간단한 대화 루프 구현
# 목적: Function Calling 전체 흐름을 하나로 통합
# 힌트:
#   1. 10개 함수 모두 tools에 등록
#   2. 자동 함수 호출 모드 사용
#   3. while 루프로 대화 반복 ("quit" 입력 시 종료)
# 예상: 다양한 질문에 적절한 함수가 호출되어 응답

def chat_with_tools():
    """Function Calling을 활용한 대화형 챗봇"""
    config = types.GenerateContentConfig(
        tools=list(AVAILABLE_FUNCTIONS.values()),  # 모든 함수 등록
        system_instruction="당신은 유용한 AI 비서입니다. 사용자의 질문에 적절한 도구를 사용해 답변하세요."
    )
    
    print("🤖 Function Calling 챗봇입니다. 'quit'을 입력하면 종료됩니다.")
    print("   사용 가능한 기능: 날씨, 주가, 팁 계산, 상품 검색, 환율, 시간, 번역, BMI, 리마인더, 뉴스")
    print()
    
    # TODO: while 루프로 대화 구현
    # while True:
    #     user_input = input("You: ")
    #     if user_input.lower() == "quit":
    #         break
    #     ...

# chat_with_tools()  # 실행하려면 주석 해제

---
## 7. 정리

### 핵심 요약

| 개념 | 설명 |
|-----|------|
| 자동 호출 | `tools=[함수]` → SDK가 자동 실행 |
| 수동 호출 | `automatic_function_calling=disable` → 개발자가 직접 실행 |
| AUTO 모드 | 모델이 함수 호출 여부 판단 |
| ANY 모드 | 반드시 함수 호출 |
| NONE 모드 | 함수 호출 금지 |

### 다음 단계
→ `03_gemini_ftn_calling_build__intermediate.ipynb`에서 직접 함수 스키마 설계